In [48]:
import requests
import pandas as pd
import datetime
from datetime import datetime, timedelta
import pytz
import seaborn as sns


In [33]:
url = "https://aerodatabox.p.rapidapi.com/airports/iata/DME"

headers = {
	"X-RapidAPI-Key": "90699f41ebmsh8a8aef972097524p1bc434jsn473111efa6f3",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"icao":"UUDD","iata":"DME","localCode":"ДМД","shortName":"Domodedovo","fullName":"Moscow, Domodedovo","municipalityName":"Moscow","location":{"lat":55.4088,"lon":37.9063},"country":{"code":"RU","name":"Russia"},"continent":{"code":"EU","name":"Europe"},"timeZone":"Europe/Moscow","urls":{"webSite":"http://www.domodedovo.ru/en/","wikipedia":"https://en.wikipedia.org/wiki/Domodedovo_International_Airport","twitter":"http://twitter.com/AirportDme","googleMaps":"https://www.google.com/maps/@55.408798,37.906299,14z","flightRadar":"https://www.flightradar24.com/55.41,37.91/14"}}


In [17]:
response.json()['iata']

'DME'

In [59]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
  list_for_df = []

  for i in range(len(latitudes)):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/100/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_for_df.append(pd.json_normalize(response.json()['items']))

  return pd.concat(list_for_df, ignore_index=True)

icao_airport_codes([55.4088, 52.5200, 9.9312], [37.9063, 13.4050, 76.2673]) 

,icao,iata,localCode,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,UUDD,DME,ДМД,"Moscow, Domodedovo",Domodedovo,Moscow,RU,55.40880,37.906300
1,UUBW,ZIA,ЖУК,"Moscow, Zhukovsky",Zhukovsky,Moscow,RU,55.55330,38.150000
2,UUWW,VKO,ВНК,"Moscow, Vnukovo",Vnukovo,Moscow,RU,55.59150,37.261500
3,UUEE,SVO,ШРМ,"Moscow, Sheremetyevo",Sheremetyevo,Moscow,RU,55.97260,37.414600
4,EDDB,BER,NaN,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889
5,VOCI,COK,NaN,"Kochi, Cochin",Cochin,Kochi,IN,10.15200,76.401900


In [5]:
arrivals_c = arrivals.copy()

'EDDH' 'EGLC' 'EGCC' 'EGLC'

In [15]:
#arrivals_c['Departure Airport'].value_counts().nlargest(50)

In [11]:
arrivals_c[arrivals_c['Aircraft Model']=='Airbus A380-800']

,Arrival Airport,Flight Number,Airline Name,Arrival Time,Arrival Terminal,Departure Airport,Aircraft Model,Data Retrieval Time
218,Munich,EK 49,Emirates,06-03-2023 12:30,1C,Dubai City,Airbus A380-800,06-03-2023 08:45
371,Munich,EK 51,Emirates,06-03-2023 19:35,1C,Dubai City,Airbus A380-800,06-03-2023 08:45
526,Frankfurt-am-Main,SQ 25,Singapore,06-03-2023 09:50,1,New York,Airbus A380-800,06-03-2023 08:45
1419,Frankfurt-am-Main,EK 47,Emirates,06-03-2023 19:25,2,Dubai City,Airbus A380-800,06-03-2023 08:45


In [60]:
#cities = ['EDDB','EGCC','EDDH','EGLC']      
#cities = ['EDDB','EDDM','EDDF'] 
cities = ['EDDB']

def flight_arrivals(cities):
    arrivals = {'arrival_airport':[],
                'flight_number':[],
                'airline_name':[],
                'arrival_time':[],
                'arrival_terminal':[],
                'departure_airport':[],               
                'aircraft_model':[],
                'data_retrieval_time':[]
               }
    
    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)
    current_date_time = (now.strftime('%Y-%m-%dT%H:%M'))
    current_date_time_table = (now.strftime('%d-%m-%Y %H:%M'))
    end_time_cal = datetime.now() + timedelta(hours=12)
    end_date_time = end_time_cal.strftime('%Y-%m-%dT%H:%M')
    
    for city in cities:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{city}/{current_date_time}/{end_date_time}"
        querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

        headers = {
                    "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6",
                    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                   }

        response = requests.request("GET", url, headers=headers, params=querystring)
        
        ###########
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{city}"

        headers = {
                    "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6",
                    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

        response1 = requests.request("GET", url, headers=headers)

        
        for i in response.json()['arrivals']:
            arrivals['departure_airport'].append(i['departure']['airport']['name'])
            arrivals['airline_name'].append(i['airline']['name'])
            arrivals['flight_number'].append(i['number'])
            arrivals['arrival_time'].append((datetime.strptime((i['arrival']['scheduledTimeLocal'].split('+')[0]),'%Y-%m-%d %H:%M')).strftime('%d-%m-%Y %H:%M'))
            try:
                arrivals['arrival_terminal'].append(i['arrival']['terminal'])
            except:
                arrivals['arrival_terminal'].append('-')
                
            try:
                arrivals['aircraft_model'].append(i['aircraft']['model'])
            except:
                arrivals['aircraft_model'].append('-')
        
            arrivals['arrival_airport'].append(response1.json()['municipalityName'])
            arrivals['data_retrieval_time'].append(current_date_time_table)
            
            

    arrivals_df = pd.DataFrame.from_dict(arrivals)
    
    arrivals = arrivals_df.rename(columns={'arrival_airport': 'Arrival Airport', 
                                          'flight_number': 'Flight Number',
                                          'airline_name': 'Airline Name', 
                                          'arrival_time': 'Arrival Time', 
                                          'arrival_terminal': 'Arrival Terminal',
                                          'departure_airport': 'Departure Airport',
                                          'aircraft_model': 'Aircraft Model',
                                           'data_retrieval_time': 'Data Retrieval Time'
                                          })
    #arrivals['Arrival Time']= pd.to_datetime(arrivals['Arrival Time'])
    #arrivals['Data Retrieval Time']= pd.to_datetime(arrivals['Data Retrieval Time'])
    
    return arrivals
        
        
       
#flight_arrivals(cities)
arrivals = flight_arrivals(cities)

In [61]:
arrivals_c = arrivals.copy()

arrivals_c[arrivals_c['Aircraft Model']=='Airbus A380-800']

,Arrival Airport,Flight Number,Airline Name,Arrival Time,Arrival Terminal,Departure Airport,Aircraft Model,Data Retrieval Time
144,Manchester,EK 17,Emirates,06-03-2023 11:15,1,Dubai City,Airbus A380-800,06-03-2023 10:54
216,Manchester,EK 19,Emirates,06-03-2023 18:40,1,Dubai City,Airbus A380-800,06-03-2023 10:54


In [54]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2023-03-06T20:00/2023-03-07T08:00"
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"true","withLocation":"false"}

headers = {
         "X-RapidAPI-Key": "3c16ed817amshbef86ec42129e0bp1cefefjsn62c8e9534fe6",
         "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
         }

response = requests.request("GET", url, headers=headers, params=querystring)
        

In [55]:
response

<Response [200]>

In [ ]:
response.json()